In [1]:
import time
import requests
import dropbox
import json
import os

# Credenciales de tu aplicación Dropbox
APP_KEY = "sxz8wblgpfgjii1"
APP_SECRET = "i4zsuphuk1s2zxu"

# Archivo donde se guardarán los tokens
TOKEN_FILE = 'dropbox_token.json'

def authenticate():
    """Autentica al usuario la primera vez y guarda el token."""
    print("Autenticación inicial necesaria. Sigue las instrucciones.")
    # auth_url = (
    #     f"https://www.dropbox.com/oauth2/authorize?client_id={APP_KEY}&response_type=code"
    # )
    # print(f"Ve a este enlace para autorizar la aplicación: {auth_url}")
    auth_code = 'xCX7tY8KgZAAAAAAAAAA-xnfGv8x8kL2GDIOqHM-fZA'

    # Intercambiar el código por un token de acceso y refresco
    response = requests.post(
        "https://api.dropbox.com/oauth2/token",
        data={
            "code": auth_code,
            "grant_type": "authorization_code",
            "client_id": APP_KEY,
            "client_secret": APP_SECRET,
        },
    )

    if response.status_code == 200:
        tokens = response.json()
        tokens["expires_at"] = time.time() + tokens["expires_in"]
        save_tokens(tokens)
        print("Autenticación exitosa. Los tokens se han guardado.")
    else:
        raise Exception("Error al autenticar: ", response.json())

def save_tokens(tokens):
    """Guarda los tokens en un archivo."""
    with open(TOKEN_FILE, 'w') as f:
        json.dump(tokens, f)

def load_tokens():
    """Carga los tokens desde el archivo."""
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, 'r') as f:
            return json.load(f)
    else:
        return None

def refresh_token():
    """Renueva el token de acceso si ha expirado."""
    tokens = load_tokens()
    if not tokens:
        raise Exception("No hay tokens almacenados. Por favor, autentícate primero.")

    if tokens["expires_at"] > time.time():
        # El token aún es válido
        return tokens["access_token"]

    # El token ha expirado, renueva usando el refresh_token
    response = requests.post(
        "https://api.dropbox.com/oauth2/token",
        data={
            "grant_type": "refresh_token",
            "refresh_token": tokens["refresh_token"],
            "client_id": APP_KEY,
            "client_secret": APP_SECRET,
        },
    )

    if response.status_code == 200:
        new_tokens = response.json()
        new_tokens["refresh_token"] = tokens["refresh_token"]  # Mantener el refresh_token existente
        new_tokens["expires_at"] = time.time() + new_tokens["expires_in"]
        save_tokens(new_tokens)
        return new_tokens["access_token"]
    else:
        raise Exception("Error al renovar el token: ", response.json())

def get_access_token():
    """Obtiene un token de acceso válido, renovándolo si es necesario."""
    tokens = load_tokens()
    if not tokens:
        print("No se encontraron tokens. Autenticando por primera vez...")
        authenticate()
        tokens = load_tokens()
    return refresh_token()

# Uso del token en tu script
ACCESS_TOKEN = get_access_token()

# Función para obtener metadatos desde Dropbox
def get_epub_metadata_from_dropbox(ACCESS_TOKEN, folder_path='/Aplicaciones/Rakuten Kobo'):
    all_metadata = []
    try:
        dbx = dropbox.Dropbox(ACCESS_TOKEN)
        response = dbx.files_list_folder(path=folder_path)

        for entry in response.entries:
            if entry.name.endswith('.epub'):
                try:
                    print(f"Procesando archivo: {entry.name}")
                    metadata, res = dbx.files_download(path=f"{folder_path}/{entry.name}")
                    epub_content = res.content
                    # Aquí procesarías el archivo epub_content como necesites
                    all_metadata.append({"filename": entry.name})
                except Exception as e:
                    print(f"Error procesando {entry.name}: {e}")

        df = pd.DataFrame(all_metadata)
        return df
    except Exception as e:
        print(f"Error: {e}")

# Llama a la función con el token válido
df_epub_metadata = get_epub_metadata_from_dropbox(ACCESS_TOKEN)
print(df_epub_metadata)


No se encontraron tokens. Autenticando por primera vez...
Autenticación inicial necesaria. Sigue las instrucciones.
Autenticación exitosa. Los tokens se han guardado.
Procesando archivo: Sex_crack_Mario_Luna.epub
Procesando archivo: Encuentra_tu_persona_vitamina_Marian_Rojas_Estape.epub
Procesando archivo: Soy_introvertido_Y_que_Steve_Allen.epub
Procesando archivo: Scar_Tissue_Anthony_Kiedis (1).epub
Procesando archivo: PNL_Tecnicas_prohibidas_de_persuasion_Steve_Allen.epub
Procesando archivo: OPEN_Andre_Agassi.epub
Procesando archivo: No_More_Mr._Nice_Guy.epub
Procesando archivo: The_Rational_Male.epub
Procesando archivo: Matches_The_Tinder_Book_For_Men.epub
Procesando archivo: El_lobo_de_Wall_Street_Jordan_Belfort.epub
Procesando archivo: El_guardian_invisible_Dolores_Redondo.epub
Procesando archivo: Steve_Jobs_Walter_Isaacson.epub
Procesando archivo: Bobby_Fischer_El_Ajedrez_es_la_vida_E_J_Rodriguez.epub
Procesando archivo: RAFA_Mi_historia_Rafael_Nadal.epub
Procesando archivo: Once